# TODO: Zklend

In [1]:
from typing import Dict
import collections
import decimal

import IPython.display
import pandas

import constants
import db

## Load and prepare events

In [2]:
# Establish the connection.
connection = db.establish_connection()

In [3]:
# Load all Zklend events.
zklend_events = pandas.read_sql(
    sql =  """
        SELECT
            *
        FROM
            starkscan_events
        WHERE
            from_address='{}';
    """.format(constants.Protocol.ZKLEND.value),
    con = connection,
)

/tmp/ipykernel_164690/2890827712.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  zklend_events = pandas.read_sql(


In [4]:
# Close the connection.
connection.close()

In [7]:
# TODO: move to constants?
RELEVANT_EVENTS = {
    'Deposit',
    'Withdrawal',
    'CollateralEnabled',
    'CollateralDisabled',
    'Borrowing',
    'Repayment',
    'Liquidation',
    'AccumulatorsSync',
    'InterestRatesSync',
}

# Remove redundant data, sort remaining data and set index.
zklend_events.drop(
    zklend_events[~zklend_events['key_name'].isin(RELEVANT_EVENTS)].index,
    inplace = True,
)
zklend_events.sort_values('block_number', inplace = True)
zklend_events.set_index('id', inplace = True)

# Process events

Events examples:
- [Deposit](https://starkscan.co/event/0x036185142bb51e2c1f5bfdb1e6cef81f8ea87fd4d777990014249bf5435fd31b_3)
- [InterestRatesSync](https://starkscan.co/event/0x06fa3dd6e12c9a66aeacd2eefa5a2ff2915dd1bb4207596de29bd0e8cdeeae66_3)
- [AccumulatorsSync](https://starkscan.co/event/0x029628b89875a98c1c64ae206e7eb65669cb478a24449f3485f5e98aba6204dc_0)
- [CollateralEnabled](https://starkscan.co/event/0x036185142bb51e2c1f5bfdb1e6cef81f8ea87fd4d777990014249bf5435fd31b_6)
- [CollateralDisabled](https://starkscan.co/event/0x0049b445bed84e0118795dbd22d76610ccac2ad626f8f04a1fc7e38113c2afe7_0)
- [Withdrawal](https://starkscan.co/event/0x03472cf7511687a55bc7247f8765c4bbd2c18b70e09b2a10a77c61f567bfd2cb_4)
- [Borrowing](https://starkscan.co/event/0x076b1615750528635cf0b63ca80986b185acbd20fa37f0f2b5368a4f743931f8_3)
- [Repayment](https://starkscan.co/event/0x06fa3dd6e12c9a66aeacd2eefa5a2ff2915dd1bb4207596de29bd0e8cdeeae66_5)
- [Liquidation](https://starkscan.co/event/0x07b8ec709df1066d9334d56b426c45440ca1f1bb841285a5d7b33f9d1008f256_5)

In [7]:
# # TODO: investigate changing lending rates within a single block  
# def huhu(data):
#     unique_events = set(data['key_name'].unique())
#     if {'Deposit', 'Withdrawal'} - unique_events:
#         return

#     deposit_transaction_hashes = data.loc[data['key_name'] == 'Deposit', 'transaction_hash'].unique()
#     withdrawal_transaction_hashes = data.loc[data['key_name'] == 'Withdrawal', 'transaction_hash'].unique()
#     deposit_irs_data = data.loc[
#         (data['key_name'] == 'InterestRatesSync')
#         & (data['transaction_hash'].isin(deposit_transaction_hashes)),
#         'data',
#     ]
#     deposit_lending_rates = [int(x[1], base = 16) / decimal.Decimal('1e27') for x in deposit_irs_data]
#     withdrawal_irs_data = data.loc[
#         (data['key_name'] == 'InterestRatesSync')
#         & (data['transaction_hash'].isin(withdrawal_transaction_hashes)),
#         'data',
#     ]
#     withdrawal_lending_rates = [int(x[1], base = 16) / decimal.Decimal('1e27') for x in withdrawal_irs_data]
#     max_deposit_lending_rate = max(deposit_lending_rates)
#     min_withdrawal_lending_rate = min(withdrawal_lending_rates)
#     print(data['block_number'].iloc[0], min_withdrawal_lending_rate > max_deposit_lending_rate)

# # zklend_events
# zklend_events.groupby('block_number').apply(huhu)

In [89]:
# # TODO
# for _, event in zklend_events.iterrows():
#     # 0x887df661288558755d0bed2561fb0cbe708c3006d6f3ecf4ddc6024ef663b3
#     if '0x5483e462b33039d404e697da4b0335d5c9d7a52c27df3ae861201a421d76476' in event['data']:
#         print(event['key_name'], event['data'], event)
#         print('-----')

In [90]:
# # TODO: al should be used!
# #il
# print(int('0x90170b0f0bfc5ed', base = 16) / (1 + 935797134342497298432550 / 1e27))
# print(int('0x90170b1823bf2d7', base = 16) / (1 + 933058669202001800242948 / 1e27))
# #ib
# print(int('0x90170b0f0bfc5ed', base = 16) / (1 + 10731963097172697163313961 / 1e27))
# print(int('0x90170b1823bf2d7', base = 16) / (1 + 10716248879884742150516333 / 1e27))

# #al
# print(int('0x90170b0f0bfc5ed', base = 16) / (1000141824517080582534436087 / 1e27))
# print(int('0x90170b1823bf2d7', base = 16) / (1000141828278971777682292138 / 1e27))
# #ad
# print(int('0x90170b0f0bfc5ed', base = 16) / (1001660803727491078914793512 / 1e27))
# print(int('0x90170b1823bf2d7', base = 16) / (1001660854709175287450155418 / 1e27))

# print('-----')

# print(1000141824517080582534436087 / 1e27)
# print(1000141828278971777682292138 / 1e27)

In [103]:
# TODO: investigate changing lending rates within a single block  
def huhu(data):
    unique_events = set(data['key_name'].unique())
    if {'Deposit'} - unique_events:
        return

    if data.loc[data['key_name'] == 'Deposit', 'event_index'].iloc[0] < data.loc[data['key_name'] == 'AccumulatorsSync', 'event_index'].iloc[0]:
        print(
            data.loc[data['key_name'] == 'Deposit', 'event_index'].iloc[0],
            data.loc[data['key_name'] == 'AccumulatorsSync', 'event_index'].iloc[0],
        )

zklend_events.groupby('block_number').apply(huhu)

""


In [12]:
class AccumulatorState:
    '''
    TODO
    '''

    def __init__(self) -> None:
        self.lending_accumulator: decimal.Decimal = decimal.Decimal('1e27')
        self.debt_accumulator: decimal.Decimal = decimal.Decimal('1e27')

    def accumulators_sync(self, lending_accumulator: int, debt_accumulator: int):
        self.lending_accumulator = decimal.Decimal(str(lending_accumulator)) / decimal.Decimal('1e27')
        self.debt_accumulator = decimal.Decimal(str(debt_accumulator)) / decimal.Decimal('1e27')


class InterestRatesState:
    '''
    TODO
    '''

    def __init__(self) -> None:
        self.lending_rate: decimal.Decimal = decimal.Decimal('1e27')
        self.borrowing_rate: decimal.Decimal = decimal.Decimal('1e27')

    def interest_rates_sync(self, lending_rate: int, borrowing_rate: int):
        self.lending_rate = decimal.Decimal(str(lending_rate)) / decimal.Decimal('1e27')
        self.borrowing_rate = decimal.Decimal(str(borrowing_rate)) / decimal.Decimal('1e27')


class UserTokenState:
    '''
    TODO

    We are making a simplifying assumption that when collateral is enabled, all
    deposits of the given token are considered as collateral.
    '''

    def __init__(self, token: str) -> None:
        self.token: str = token
        # TODO: keep track of avg deposit age?
        self.deposit: int = 0
        self.collateral_enabled: bool = False
        # TODO: keep track of avg borrowings age? or we don't need to?
        self.borrowings: int = 0


class UserState:
    '''
    TODO
    '''

    def __init__(self) -> None:
        self.token_states: Dict[str, UserTokenState] = {
            'ETH': UserTokenState('ETH'),
            'wBTC': UserTokenState('wBTC'),
            'USDC': UserTokenState('USDC'),
            'DAI': UserTokenState('DAI'),
            'USDT': UserTokenState('USDT'),
        }
        self.health_factor: float = 1.0  # TODO: is this a good default value??

    def deposit(self, token: str, face_amount: int):
        self.token_states[token].deposit += face_amount

    def withdrawal(self, token: str, face_amount: int):
        self.token_states[token].deposit -= face_amount

    def collateral_enabled(self, token: str):
        self.token_states[token].collateral_enabled = True

    def collateral_disabled(self, token: str):
        self.token_states[token].collateral_enabled = False

    def borrowing(self, token: str, raw_amount: int, face_amount: int):
        self.token_states[token].borrowings += raw_amount

    def repayment(self, token: str, raw_amount: int, face_amount: int):
        self.token_states[token].borrowings -= raw_amount

    def liquidation(
        self,
        debt_token: str,
        debt_raw_amount: int,
        debt_face_amount: int,
        collateral_token: int,
        collateral_amount: int,
    ):
        self.token_states[debt_token].borrowings -= debt_raw_amount
        self.token_states[collateral_token].deposit -= collateral_amount


class State:
    '''
    TODO
    '''

    EVENTS_FUNCTIONS_MAPPING: Dict[str, str] = {
        'Deposit': 'process_deposit_event',
        'Withdrawal': 'process_withdrawal_event',
        'CollateralEnabled': 'process_collateral_enabled_event',
        'CollateralDisabled': 'process_collateral_disabled_event',
        'Borrowing': 'process_borrowing_event',
        'Repayment': 'process_repayment_event',
        'Liquidation': 'process_liquidation_event',
        'AccumulatorsSync': 'process_accumulators_sync_event',
        'InterestRatesSync': 'process_interest_rates_sync_event',
    }

    def __init__(self) -> None:
        self.user_states: collections.defaultdict = collections.defaultdict(UserState)
        self.accumulator_states: Dict[str, AccumulatorState] = {
            'ETH': AccumulatorState(),
            'wBTC': AccumulatorState(),
            'USDC': AccumulatorState(),
            'DAI': AccumulatorState(),
            'USDT': AccumulatorState(),
        }
        self.interest_rates_states: Dict[str, InterestRatesState] = {
            'ETH': InterestRatesState(),
            'wBTC': InterestRatesState(),
            'USDC': InterestRatesState(),
            'DAI': InterestRatesState(),
            'USDT': InterestRatesState(),
        }

    def process_event(self, event: pandas.Series) -> None:
        name = event['key_name']
        getattr(self, self.EVENTS_FUNCTIONS_MAPPING[name])(event = event)

    def process_deposit_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `user`, `token`, `face_amount`.
        user = event['data'][0]
        token = constants.get_symbol(event['data'][1])
        # TODO: divide by something or store like this?
        face_amount = int(event['data'][2], base = 16)
        # TODO: sanity checks/asserts?
#         # TODO
        if user == '0x4a587e33647247ed71d178b25155a27463765f7c113f05904126692d348cab1':
#         if user == '0x4a587e33647247ed71d178b25155a27463765f7c113f05904126692d348cab1' and token == 'USDT':  # LA: USDT
#         if user == '0x51ca7baac15c5b6439a87b1db31bcb7c42db018cd025a4b8303424cdfcd2688' and token == 'DAI':  # LA: DAI
#         if user == '0x4066d4927b6a5e3a7100819c7e722e8239cfcade7d763f9e318e58ba44213a8' and token == 'USDC':  # LA: USDC
#         if user == '0x51c794946b7e4a351953c464936e475e286e2a6747a43d91867a5ad85622a4c' and token == 'ETH':  # LA: ETH
#         if user == '0x1788e38740ee98a0da67a503c3eb3ac7813b3b2fb0367c492acb3296adc80dc' and token == 'ETH':  # original
            print(
                'de',
                token,
                self.accumulator_states[token].lending_accumulator,
#                 1 + self.interest_rates_states[token].lending_rate,
                face_amount,
            )
#         # TODO: use raw_amount?
        raw_amount = face_amount / self.accumulator_states[token].lending_accumulator
#         raw_amount = face_amount / (1 + self.interest_rates_states[token].lending_rate)
        self.user_states[user].deposit(token = token, face_amount = raw_amount)

    def process_withdrawal_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `user`, `token`, `face_amount`.
        user = event['data'][0]
        token = constants.get_symbol(event['data'][1])
        face_amount = int(event['data'][2], base = 16)
#         # TODO
        if user == '0x4a587e33647247ed71d178b25155a27463765f7c113f05904126692d348cab1':
#         if user == '0x4a587e33647247ed71d178b25155a27463765f7c113f05904126692d348cab1' and token == 'USDT':  # LA: USDT
#         if user == '0x51ca7baac15c5b6439a87b1db31bcb7c42db018cd025a4b8303424cdfcd2688' and token == 'DAI':  # LA: DAI
#         if user == '0x4066d4927b6a5e3a7100819c7e722e8239cfcade7d763f9e318e58ba44213a8' and token == 'USDC':  # LA: USDC
#         if user == '0x51c794946b7e4a351953c464936e475e286e2a6747a43d91867a5ad85622a4c' and token == 'ETH':  # LA: ETH
#         if user == '0x1788e38740ee98a0da67a503c3eb3ac7813b3b2fb0367c492acb3296adc80dc' and token == 'ETH':  # original
            print(
                'we',
                token,
                self.accumulator_states[token].lending_accumulator,
#                 1 + self.interest_rates_states[token].lending_rate,
                face_amount,
            )
#         # TODO: use raw_amount?
        raw_amount = face_amount / self.accumulator_states[token].lending_accumulator
#         raw_amount = face_amount / (1 + self.interest_rates_states[token].lending_rate)
        self.user_states[user].withdrawal(token = token, face_amount = raw_amount)

    def process_collateral_enabled_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `user`, `token`.
        user = event['data'][0]
        token = constants.get_symbol(event['data'][1])
        self.user_states[user].collateral_enabled(token = token)

    def process_collateral_disabled_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `user`, `token`.
        user = event['data'][0]
        token = constants.get_symbol(event['data'][1])
        self.user_states[user].collateral_disabled(token = token)

    def process_borrowing_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `user`, `token`, `raw_amount`, `face_amount`.
        user = event['data'][0]
        token = constants.get_symbol(event['data'][1])
        raw_amount = int(event['data'][2], base = 16)
        face_amount = int(event['data'][3], base = 16)  # TODO: relevant?
        self.user_states[user].borrowing(
            token = token,
            raw_amount = raw_amount,
            face_amount = face_amount,
        )
#         # TODO
        if user == '0x4a587e33647247ed71d178b25155a27463765f7c113f05904126692d348cab1':
            print(
                'bore',
                token,
                raw_amount,
            )

    def process_repayment_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `repayer`, `beneficiary`, `token`, `raw_amount`,
        # `face_amount`.
        repayer = event['data'][0]  # TODO: relevant?
        beneficiary = event['data'][1]
        token = constants.get_symbol(event['data'][2])
        raw_amount = int(event['data'][3], base = 16)
        face_amount = int(event['data'][4], base = 16)  # TODO: relevant?
        self.user_states[beneficiary].repayment(
            token = token,
            raw_amount = raw_amount,
            face_amount = face_amount,
        )
#         # TODO
        if beneficiary == '0x4a587e33647247ed71d178b25155a27463765f7c113f05904126692d348cab1':
            print(
                'repe',
                token,
                raw_amount,
                repayer,
                beneficiary,
            )

    def process_liquidation_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `liquidator`, `user`, `debt_token`, `debt_raw_amount`,
        # `debt_face_amount`, `collateral_token`, `collateral_amount`.
        liquidator = event['data'][0]  # TODO: relevant?
        user = event['data'][1]
        debt_token = constants.get_symbol(event['data'][2])
        debt_raw_amount = int(event['data'][3], base = 16)
        debt_face_amount = int(event['data'][4], base = 16)  # TODO: relevant?
        collateral_token = constants.get_symbol(event['data'][5])
        collateral_amount = int(event['data'][6], base = 16)
#         # TODO
        if user == '0x4a587e33647247ed71d178b25155a27463765f7c113f05904126692d348cab1':
#         if user == '0x4a587e33647247ed71d178b25155a27463765f7c113f05904126692d348cab1' and collateral_token == 'USDT':  # LA: USDT
#         if user == '0x51ca7baac15c5b6439a87b1db31bcb7c42db018cd025a4b8303424cdfcd2688' and collateral_token == 'DAI':  # LA: DAI
#         if user == '0x4066d4927b6a5e3a7100819c7e722e8239cfcade7d763f9e318e58ba44213a8' and collateral_token == 'USDC':  # LA: USDC
#         if user == '0x51c794946b7e4a351953c464936e475e286e2a6747a43d91867a5ad85622a4c' and collateral_token == 'ETH':  # LA: ETH
#         if user == '0x1788e38740ee98a0da67a503c3eb3ac7813b3b2fb0367c492acb3296adc80dc' and collateral_token == 'ETH':  # original
            print(
                'le',
                collateral_token,
                self.accumulator_states[collateral_token].lending_accumulator,
#                 1 + self.interest_rates_states[collateral_token].lending_rate,
                collateral_amount,
            )
#         # TODO: use raw_amount?
        collateral_raw_amount = collateral_amount / self.accumulator_states[collateral_token].lending_accumulator
#         collateral_raw_amount = collateral_amount / (1 + self.interest_rates_states[collateral_token].lending_rate)
        self.user_states[user].liquidation(
            debt_token = debt_token,
            debt_raw_amount = debt_raw_amount,
            debt_face_amount = debt_face_amount,
            collateral_token = collateral_token,
            collateral_amount = collateral_raw_amount,
        )

    def process_accumulators_sync_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `token`, `lending_accumulator`, `debt_accumulator`.
        token = constants.get_symbol(event['data'][0])
        lending_accumulator = int(event['data'][1], base = 16)
        debt_accumulator = int(event['data'][2], base = 16)
        self.accumulator_states[token].accumulators_sync(
            lending_accumulator = lending_accumulator,
            debt_accumulator = debt_accumulator,
        )

    def process_interest_rates_sync_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `token`, `lending_rate`, `borrowing_rate`.
        token = constants.get_symbol(event['data'][0])
        lending_rate = int(event['data'][1], base = 16)
        borrowing_rate = int(event['data'][2], base = 16)
        self.interest_rates_states[token].interest_rates_sync(
            lending_rate = lending_rate,
            borrowing_rate = borrowing_rate,
        )

In [13]:
# Iterate over ordered events to obtain the final state of each user.
state = State()
zklend_events.sort_values(['block_number', 'event_index'], inplace = True)
for _, event in zklend_events.iterrows():
    state.process_event(event = event)

de wBTC 1.000000006140944823213094689 350000
bore DAI 49994578800743128422
repe DAI 49994578800743128422 0x4a587e33647247ed71d178b25155a27463765f7c113f05904126692d348cab1 0x4a587e33647247ed71d178b25155a27463765f7c113f05904126692d348cab1
we wBTC 1.000000006262874011094123863 349964
de wBTC 1.000000008218724182525946076 349964
bore ETH 27654464334982692
we wBTC 1.002609841769984290894381720 84444
le wBTC 1.002631402202959243142387213 2479
de DAI 1.006162760513697239474853707 120909957429278633911
we USDT 1.000464830532474829326215190 5000000


In [ ]:
# TODO: LA: USDT: only withdrawal???

In [144]:
# # TODO: LA: DAI: 3.7e-21
# print(
#     (decimal.Decimal('101520981396932145103') / decimal.Decimal('1.007523582221253501945848711'))
#     - (decimal.Decimal('10050618469516417327') / decimal.Decimal('1.007527763745309229687257401'))
#     - (decimal.Decimal('91470786964259811214') / decimal.Decimal('1.007527793423698464487712176'))
# )
# print(0.37138058 / 101520981396932145103)

In [141]:
# # TODO: LA: USDC: 4.4e-8
# print(
#     (decimal.Decimal('31930594') / decimal.Decimal('1.000013873233819040181896554'))
#     - (decimal.Decimal('319274') / decimal.Decimal('1.000013887902793865065972643'))
#     - (decimal.Decimal('3160816') / decimal.Decimal('1.000013955687004108664016957'))
#     - (decimal.Decimal('28450509') / decimal.Decimal('1.000013990710252631470246696'))
# )
# print(1.39246404407114132779 / 31930594)

In [134]:
# # TODO: LA: ETH: 1.5e-8
# print(
#     (decimal.Decimal('20526057148959784') / decimal.Decimal('1.000067721521951187025058299'))
#     - (decimal.Decimal('6446294485033218') / decimal.Decimal('1.000076223539147832087156482'))
#     - (decimal.Decimal('4876157887318462') / decimal.Decimal('1.000101302797776715219460882'))
#     - (decimal.Decimal('9204214421113239') / decimal.Decimal('1.000110186174027260459949047'))
# )
# print(300437786.967166206714 / 20526057148959784)

# TODO: Debugging negative deposits below

In [107]:
# # TODO: lending_accumulator
# print(
#     (900000000000000000000)
#     + (80000000000000000000)
#     + (502493389030314091303)
#     - (1443577111469088876324)
#     - (43000000000000000000)
# )
# print(
#     (900000000000000000000 / 1.000000000000000000000000000)
#     + (80000000000000000000 / 1.000341640805821649966147094)
#     + (502493389030314091303 / 1.006022702795017542366777716)
#     - (1443577111469088876324 / 1.007697118768125486105104087)
#     - (43000000000000000000 / 1.007707517677641512709962394)
# )
# print(
#     (900000000000000000000 * 1.000000000000000000000000000)
#     + (80000000000000000000 * 1.000341640805821649966147094)
#     + (502493389030314091303 * 1.006022702795017542366777716)
#     - (1443577111469088876324 * 1.007697118768125486105104087)
#     - (43000000000000000000 * 1.007707517677641512709962394)
# )

In [108]:
# TODO: negative deposits?
for attr in ['deposit', 'borrowings']:
    for token in ['ETH', 'wBTC', 'USDC', 'DAI', 'USDT']:
        values = [getattr(user_state.token_states[token], attr) for user, user_state in state.user_states.items()]
        print(attr, token, pandas.Series(values).min())

deposit ETH -300437786.967166206714
deposit wBTC 0
deposit USDC -1.39246404407114132779
deposit DAI -0.37138058
deposit USDT -4997676.927172805096183238170
borrowings ETH 0
borrowings wBTC 0
borrowings USDC 0
borrowings DAI 0
borrowings USDT 0


In [145]:
# TODO: show negative deposits
[
    (user, user_state.token_states['USDT'].deposit)
    for user, user_state
    in state.user_states.items()
    if user_state.token_states['USDT'].deposit < 0
]

[('0x30b399e06903676ada3eccd5522e0cca4c4ad0101468c0ac407a56aa1a0ed3c',
  Decimal('-0.898528111921624881')),
 ('0x4a587e33647247ed71d178b25155a27463765f7c113f05904126692d348cab1',
  Decimal('-4997676.927172805096183238170')),
 ('0x575a7f1f50d834b4a3d18b543adfca487f8a7d4d9d3e26c512f9d714912f35a',
  Decimal('-0.84824947382307314105')),
 ('0x77af3e6973ecc448b8f45ca1127fc7d58619e74a74872b462b816ca96a370ed',
  Decimal('-0.15416214948141902336')),
 ('0x64f2207dfd9fe6d6cdd17b9501ebcbc5d32e94707e5117204d5a113948f3b45',
  Decimal('-0.2645569408314521223793011')),
 ('0x5e1d26d555f2136863478f7af53676af71659510380dd2df1542920c44992d6',
  Decimal('-0.974623600083230677698'))]

In [ ]:
# TODO: Get token prices here: https://www.coingecko.com/en/api/documentation